In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [60]:
filename = 'test'
data = load_orderbook_snapshot(filename, last_line=200)

100%|██████████| 100/100 [00:04<00:00, 21.47it/s]

Loaded 100 orderbooks from file 'test'.


In [4]:
orderbook = data[0]
print(orderbook)

orderbook2 = data[1]

diff = orderbook2.subtract(other=orderbook)  #.to_DataFrame(5)
#new = orderbook2.add(diff)

print(diff)

OrderbookContainer from 2016-11-08T10:00
  499 bids (best: 705.0)
  407 asks (best: 705.45)
  kind: 'orderbook'
OrderbookContainer from 2016-11-08T10:00
  23 bids (best: 673.62)
  27 asks (best: 705.45)
  kind: 'diff'


In [5]:
orderbook = data[0]
orderbook2 = data[1]

diff = orderbook.subtract(orderbook2)

display(diff.head())
print(diff)
ots = OrderbookTradingSimulator()
info = ots.check_current_value(orderbook, 4)
display(info)

,Amount,Type
678.31,-0.510201,bid
674.59,0.546871,bid
673.62,-0.545428,bid
705.45,-1.876972,ask
706.41,-0.045817,ask
706.81,-7.529719,ask


OrderbookContainer from 2016-11-08T10:01
  23 bids (best: 673.62)
  27 asks (best: 705.45)
  kind: 'diff'


{'cashflow': -2823.2013944685, 'trade_summary':           Amount
 705.45  2.772181
 706.17  0.052838
 706.19  0.645323
 706.89  0.105160
 707.18  0.424498, 'volume': 4.0}

In [24]:
orderbook.tail()

,Amount,Type
614.000000,1.838163,bid
614.320000,0.034763,bid
614.760000,0.000048,bid
705.224976,NaN,center
809.090000,0.200000,ask
810.000000,0.286798,ask
811.000000,0.000100,ask


In [61]:
ots = OrderbookTradingSimulator()
vol = 40
t = 10
for i in range(10):
    res = ots.trade_timespan(orderbooks=data[t*i:t*(i+1)], volume=vol, limit=707, timespan=10, verbose=True)
    vol = ots.history.volume_left.values[-1]
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(ots.history)
display(ots.buy_history)
display(res.head(5))

Traded 14.8370/40.0000 shares for -10477.6954664, 25.1630 shares left
Traded 0.0000/25.1630 shares for 0, 25.1630 shares left
Traded 0.0000/25.1630 shares for 0, 25.1630 shares left
Traded 4.6358/25.1630 shares for -3271.51483807, 20.5273 shares left
Traded 16.5880/20.5273 shares for -11720.8034537, 3.9393 shares left
No shares left at t=1 (self.t=52), Done!
Traded 3.9393/3.9393 shares for -2781.45292293, 0.0000 shares left


,ASK,BID,CENTER,LIMIT,SPREAD,TIMESPAN,VOLUME,avg,cashflow,cost,cost_avg,high,low,volume_left,volume_traded
2016-11-08T10:00,705.45,705.00,705.224976,707,0.45,10,40.000000,706.18812,-10477.695466,14.290085,0.963144,706.89,705.000000,25.163025,14.836975
2016-11-08T10:10,707.15,705.45,706.299659,707,1.70,10,25.163025,NaN,0.000000,0.000000,NaN,0.00,inf,25.163025,0.000000
2016-11-08T10:20,707.10,705.45,706.274679,707,1.65,10,25.163025,NaN,0.000000,0.000000,NaN,0.00,inf,25.163025,0.000000
2016-11-08T10:30,707.10,704.44,705.769165,707,2.66,10,25.163025,705.71401,-3271.514838,2.267055,0.489034,707.00,703.800000,20.527273,4.635752
2016-11-08T10:40,706.19,705.45,705.819935,707,0.74,10,20.527273,706.58393,-11720.803454,22.542312,1.358954,706.83,703.630000,3.939289,16.587985
2016-11-08T10:50,707.03,705.45,706.239705,707,1.58,10,3.939289,706.08000,-2781.452923,3.368186,0.855024,706.08,706.080000,0.000000,3.939289


,Amount
703.63,1.000000e-08
703.80,9.245265e-01
705.00,1.200000e-05
705.45,6.458106e+00
706.08,3.939289e+00
706.17,5.283784e-02
706.18,4.487666e+00
706.19,4.468270e+00
706.41,4.581742e-02
706.45,2.870490e+00


,Amount,Type
704.400000,1.500000,bid
704.580000,1.664360,bid
704.790000,1.600500,bid
705.120000,1.695420,bid
705.450000,0.400000,bid
706.274679,NaN,center
707.100000,0.000950,ask
707.300000,0.001210,ask
707.500000,0.000430,ask
707.600000,0.000110,ask
